In [24]:
import pandas as pd
from pymongo import MongoClient
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output, State
from functools import lru_cache
import base64

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Adjust the connection string if necessary
db = client['AAC']
collection = db['animals']

# Retrieve data for display
data = collection.find()
df = pd.DataFrame(list(data))

# Check if '_id' column exists before dropping it to avoid KeyError
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Setting up the Dash app
app = Dash(__name__)

# Load image for the logo
image_filename = r'D:\CS 340 ClientServer Development\Week 7\Grazioso Salvare Dashboard\Grazioso Salvare.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define caching mechanism for frequently accessed data
@lru_cache(maxsize=32)
def get_filtered_data(filter_type):
    if filter_type == 'WR':
        return pd.DataFrame(list(collection.find({"breed": {"$regex": "Labrador Retriever|Newfoundland", "$options": "i"}})))
    elif filter_type == 'MWR':
        return pd.DataFrame(list(collection.find({"breed": {"$regex": "German Shepherd|Border Collie", "$options": "i"}})))
    elif filter_type == 'DIT':
        # Broad match for Disaster or Individual Tracking including both dogs and cats
        return pd.DataFrame(list(collection.find({
            "$and": [
                {"animal_type": {"$in": ["Dog", "Cat"]}},  # Include both dogs and cats
                {"$or": [
                    {"outcome_subtype": {"$regex": "Disaster|Tracking", "$options": "i"}},
                    {"outcome_type": {"$regex": "Disaster|Tracking", "$options": "i"}},
                    {"breed": {"$regex": "Bloodhound|Beagle|Siamese|Persian", "$options": "i"}}  # Example breeds for tracking, including cats
                ]}
            ]
        })))
    return df

# UI Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Hr(),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'}),
        html.P("Created by: Zaffar Shiekh"),
    ]),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value': 'ALL'}
            ],
            value='ALL',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="multi",
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[], 
        selected_rows=[], 
        page_action="native", 
        page_current=0, 
        page_size=10, 
        style_table={'overflowX': 'auto'},
        style_data={'backgroundColor': 'RoyalPurple', 'color': 'white'},  # Background color for data cells
        css=[{
            'selector': '.dash-filter input',
            'rule': 'text-transform: lowercase;'
        }],
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
], style={'backgroundColor': 'RoyalPurple'})  # Background color for the entire dashboard

# Callback to update the data table based on filter type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_datatable(filter_type):
    df_filtered = get_filtered_data(filter_type)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Run the Dash server in inline mode for Jupyter
app.run_server(mode='inline')
